In [1]:
import requests
from bs4 import BeautifulSoup
from IPython.core.display import HTML
from fastcore.all import *
import pandas as pd
import json

import json
from urllib.request import urlopen

In [2]:
main_url = 'https://help.dyalog.com/18.2/Content/Language/Introduction/Language%20Elements.htm?tocpath=Language%20Reference%20Guide%7CSymbols%7C_____1'
content = requests.get(main_url)
content.encoding = content.apparent_encoding

main=BeautifulSoup(content.text)

symbol_tags = main.find_all('a',attrs={'href':lambda x: x.startswith('../Symbols/')})
symbols_dict = {o.text:o['href'] for o in symbol_tags}

# General Cards from Dyalog Docs

In [3]:
base = 'https://help.dyalog.com/18.2/Content/Language'

In [4]:
def get_hyperlink(url,url_type):
    return f" <br/><br/><a href='{url}'>Take me to {url_type}</a>"

In [5]:
cards = []
for i,sym in enumerate(L(*symbols_dict.keys())):
    doc_url = symbols_dict[sym]
    url1 = base + doc_url[2:]

    content = requests.get(url1)
    content.encoding = content.apparent_encoding
    doc_html = BeautifulSoup(content.text)

    doc_h5s = doc_html.find_all('h5')
    doc_h3 = doc_html.find('td').find('a',attrs={'name':lambda o: not o.startswith('kanchor')})
    
    
    if doc_html.find('td').text.strip() == '':
        _2 = doc_h3['name']
    else: _2 = doc_html.find('td').text.strip()
    
    # Name of glyph -> Symbol
    cards.append(pd.DataFrame([[_2 + "<br/><br/>What does this glyph look like?",
                                sym + get_hyperlink(url1,"symbol documentation"),
                               "Name->Sym"
                               ]]))
    urls = L(url1)
    for doc_h5 in doc_h5s:
        
        p = doc_h5.find_next_sibling('p')
        dy_or_mon_name = ''.join(re.findall('Dyadic|Monadic',doc_h5.text)) + ': ' + p.text.strip()
        
        # Mondadic/Dyadic Name -> Symbol
        cards.append(pd.DataFrame([[dy_or_mon_name + "<br/><br/>What does this glyph look like?",
                                    sym + get_hyperlink(url1,"symbol documentation"),
                                   "Name->Sym"
                                   ]]))
        _2  = _2 + ' <br/> ' + dy_or_mon_name 
        
        
        url_end = [o.find('a',attrs={'href':True}) for o in doc_h5.find_next_siblings('p') if o.find('a',attrs={'href':True}) is not None][0]['href']
#         url2 = base + p.find('a',attrs={'href':True})['href'][2:]
        url2 = base + url_end[2:]
        urls.append(url2)
        
        content = requests.get(url2)
        content.encoding = content.apparent_encoding
        doc_html = BeautifulSoup(content.text)
    
        try:
            desc = [o.text for o in doc_html.find('td').find_all_next('p') if o.text.strip() != '']

            documentation = ' <br/> '.join(desc)
            
            # Functional documentation -> Symbol
            cards.append(pd.DataFrame([[documentation + "<br/><br/>What does this glyph look like?",
                                        sym + get_hyperlink(url1,"symbol documentation") + get_hyperlink(url2,"primitive function documentation"),
                                       "Docs->Sym"
                                       ]]))
        except:
            print(i)
        try:
            doc_name = doc_html.find('td').find('a',attrs={'name':lambda o: not o.startswith('kanchor')})
            if doc_name['name'] not in _2:
                # Documentation Name -> Symbol
                cards.append(pd.DataFrame([[doc_name['name'] + "<br/><br/>What does this glyph look like?",
                                            sym + get_hyperlink(url1,"symbol documentation") + get_hyperlink(url2,"primitive function documentation"),
                                           "Name->Sym"
                                           ]]))

            # Documentation Name -> Functional Documentation
            # Functional Documentation -> Documentation Name
            cards.append(pd.DataFrame([
                    [doc_name['name'] + "<br/><br/>What does this glyph do?",
                     documentation + get_hyperlink(url1,"symbol documentation") + get_hyperlink(url2,"primitive function documentation"),
                     "Name->Docs"
                    ],
                    [documentation + "<br/><br/>What is the name of this glyph?",
                     doc_name['name'] + get_hyperlink(url1,"symbol documentation") + get_hyperlink(url2,"primitive function documentation"),
                    "Docs->Name"
                    ],
                               
                    ]))
        except:
            print(i)
    
    # Symbol -> Functional Documentation
    cards.append(pd.DataFrame([[sym + "<br/><br/>What does this glyph do?",
                                        documentation + ''.join([get_hyperlink(o,"Link") for o in urls.unique()]),
                                       "Sym->Docs"
                                       ]]))
    # Symbol -> Names
    cards.append(pd.DataFrame([[sym + "<br/><br/>What is this glyph called?  What's the Monadic/Dyadic names?",
                                _2 + get_hyperlink(url1,"symbol documentation"),
                               "Sym->Docs"]]))

65
72
72
72


In [6]:
df = pd.concat(cards)
df.to_csv('DyalogDocs_AllCards.csv',index=False,header=False)
df.sample(15)

,0,1,2
0,.<br/><br/>What is this glyph called? What's ...,Dot <br/> DyadicDyadic: Inner Product <br/> : ...,Sym->Docs
0,≥<br/><br/>What is this glyph called? What's ...,Greater Than Or Equal To Sign <br/> Dyadic: Is...,Sym->Docs
0,⍤<br/><br/>What does this glyph do?,Classic Edition: the symbol ⍤ is not available...,Sym->Docs
0,GradeDown(Dyadic)<br/><br/>What does this glyp...,Y must be a simple character array of rank gre...,Name->Docs
1,Y may be any numeric array. X may be any nume...,Multiply <br/><br/><a href='https://help.dyalo...,Docs->Name
0,Monadic: Table<br/><br/>What does this glyph l...,⍪ <br/><br/><a href='https://help.dyalog.com/1...,Name->Sym
0,Classic Edition: the symbol ⍸ (Iota Underbar) ...,⍸ <br/><br/><a href='https://help.dyalog.com/1...,Docs->Sym
0,X and Y may be any arrays. R is a simple Bool...,⍷ <br/><br/><a href='https://help.dyalog.com/1...,Docs->Sym
0,Each(withMonadicOperand)<br/><br/>What does th...,f may be any monadic function. Y may be any a...,Name->Docs
0,Reciprocal<br/><br/>What does this glyph do?,Y must be a numeric array. R is numeric. R i...,Name->Docs


### Stuff that doesn't work quite right
+ A few documentation pages have code cells mixed into the text.  Currently it only pulls \<p> text cells so the code is lost.  That could be fixed, but it's infrequent and not a huge deal since there's hyperlinks.
+ Fix the try/except stuff

# General Cards from study group NB

In [61]:
url = 'https://raw.githubusercontent.com/Isaac-Flath/apl-study/fastai-master/APL.ipynb'
nb = json.loads(urlopen(url).read())

In [62]:
# Define newlines on cards
def collapse_list(l): return '<br/>'.join(l) + '<br/>'

def get_hyperlink(url,url_type):
    return f" <br/><br/><a href='{url}'>Take me to {url_type}</a>"

In [63]:
def cell_filter(cell):
    cond1 = cell['cell_type'] == 'code'
    cond2 = ('outputs' in cell.keys()) and (cell['outputs'] != [])
    cond3 = cell['source'] != []
    return cond1 and cond2 and cond3

In [64]:
filtered_codes = L(nb['cells']).filter(cell_filter)

In [65]:
base = 'https://help.dyalog.com/18.2/Content/Language'

code_cards = []
for i,cell in enumerate(filtered_codes):
    _src = cell['source']
        
    _split = [[char for char in line] for line in _src]
    
    _split = [o for sublist in _split for o in sublist]
    _symbol_locs = {o:symbols_dict[o] for o in _split if o in filter_dict(symbols_dict,lambda k,v: k!='.').keys()}
    _symbol_locs

    urls = []
    
    src = collapse_list(_src)
    out = ''.join(cell['outputs'][0]['data']['text/html'])
    out = out + ''.join([get_hyperlink(base+u[2:],s) for s,u in _symbol_locs.items()])
    code_cards.append(pd.DataFrame([[src + f" <br/><br/>What does the code output?",out,"Code->Output"]]))


In [66]:
df = pd.concat(code_cards)
df.to_csv('AplStudyGroup_CodeCards.csv',index=False,header=False)
df.sample(15)

,0,1,2
0,1 2 3 ≥ 4 2 ¯1<br/> <br/><br/>What does the co...,"<span style=""white-space:pre; font-family: mon...",Code->Output
0,b<br/> <br/><br/>What does the code output?,"<span style=""white-space:pre; font-family: mon...",Code->Output
0,'A' 'C' 'F'<br/> <br/><br/>What does the code ...,"<span style=""white-space:pre; font-family: mon...",Code->Output
0,3 2 1÷2 2.5 ¯1<br/> <br/><br/>What does the co...,"<span style=""white-space:pre; font-family: mon...",Code->Output
0,'Banana' = 'a'<br/> <br/><br/>What does the co...,"<span style=""white-space:pre; font-family: mon...",Code->Output
0,3-2<br/> <br/><br/>What does the code output?,"<span style=""white-space:pre; font-family: mon...",Code->Output
0,⍳4<br/> <br/><br/>What does the code output?,"<span style=""white-space:pre; font-family: mon...",Code->Output
0,]Boxing on -style=max<br/> <br/><br/>What does...,"<span style=""white-space:pre; font-family: mon...",Code->Output
0,1<br/> <br/><br/>What does the code output?,"<span style=""white-space:pre; font-family: mon...",Code->Output
0,a<br/> <br/><br/>What does the code output?,"<span style=""white-space:pre; font-family: mon...",Code->Output
